# Generación del Modelo

Este notebook tiene la finalidad de guardar el modelo entrenado para su posterior ejecución. Una vez evaluado los modelos bajo diferentes configuraciones, estas pueden ser ingresadas enla sección de parámetros que se encuentra a continuación.

--------------------------
## Parámetros

--------------------------

### Tipos de modelo

Existen dos configuraciones para los modelos a generar. El **Modelo global** toma todos los vuelos y aerolíneas que se encuentren en el dataset para entrenar el modelo, mientras que el **Modelo Carrier** solo toma los vuelos de una aerolínea en particular, la cual debe eser específicada como parámetro de entrada.

Los nombres de los parámetros son:

- Global
- Carrier

In [1]:
model_type = 'Global'

### Características

Independiente de si se ha seleccionado el modelo global o uno por aerolínea, el modelo se entrenará y creará con tres posibles opciones que agrupan las siguientes características:

- **base**
 - LATITUDE_meters
 - LONGITUDE_meters
 - Velocity
 - Angle_V
 - dist_Tbar'
- **extend**: Las mismas características de **base** más:
 - Acceleration
 - Angle_A
- **wind**: Las mismas características de **base** más:
 - Velocity_p: Proyección del vector de la velocidad del viento sobre el vector de la velocidad del avión.

In [2]:
features = 'base'

### Opciones para guardar el modelo

Debe ingresar el nombre para el modelo y la ruta en donde será guardado. Por defecto, el nombre del modelo irá concatenado con el tipo de modelo seleccionado y las características seleccionadas, por ejemplo, **model01_Global_base.pkl**


- **save**:
 - 0: No guardar modelo
 - 1: Guardar modelo
- **name_model**
- **path_save**: ruta donde se almacenará el modelo

In [3]:
save = 1
name_model = 'model_1'
path_save = 'model/'

### Dataset

En esta sección se especifica la ruta del dataset preprocesado y dependiento del tipo de modelo seleccionado, se debe ingresar los parámetros **n_flights** ó **name_carrier**. 

_________________
**Ruta**
- **path_dataset**: Ruta donde se encuentra el dataset.

_________________
**Para modelo Global**
- **n_flights**: Número de vuelos a seleccionar del dataset.

En la sección de **Error - Random Forest Model** se muestra el número máximo de vuelos que contiene el dataset.
_________________
**Para modelo por Aerolinea**
- **name_carrier**: Nombre de la aerolinea.

In [4]:
path_dataset = 'datasets/budapest.csv'

In [5]:
# Si ha seleccionado el modelo Global debe especificar el número de vuelos.
if model_type == 'Global':
    n_flights = 300

# Si ha seleccinado el modelo Carrier debe especificar el nombre de la aerolínea.
if model_type == 'Carrier':
    name_carrier = 'WZZ'

-------------------------------

## Generación del modelo según los parámetros especificados

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_absolute_error, make_scorer, r2_score


from sklearn.externals import joblib

import warnings
warnings.filterwarnings('ignore')

-------------------------
### Funciones

-------------------------

In [7]:
%run functions_generate.py

----------------------
### Read dataset

----------------------

In [8]:
if model_type == 'Global':
    data_base = pd.read_csv(path_dataset, index_col=0)
    data_base = data_base.reset_index(drop=True)
    max_fligts = len(data_base['ID'].drop_duplicates())
    data = read_flights(data_base, n_flights)
    data = data.reset_index(drop=True)
    del data_base
    groups = data['ID']
    models_gp = data['AC_REAL_MODEL']
    callsign_gp = data['CALLSIGN']
    carrier = get_carries(callsign_gp)
    carrier = carrier.reset_index(drop=True)
elif model_type == 'Carrier':
    data = pd.read_csv(path_dataset, index_col=0)
    data = data.reset_index(drop=True)
    max_fligts = len(data['ID'].drop_duplicates())
    callsign_gp = data['CALLSIGN']
    carrier = get_carries(callsign_gp)
    carrier = carrier.reset_index(drop=True)
    data = pd.concat([data, carrier], axis=1)
    data = data.reset_index(drop=True)
    data = data[data['Carrier'] == name_carrier]
    data = data.reset_index(drop=True)
    groups = data['ID']
    models_gp = data['AC_REAL_MODEL'] 

--------------------------
### Read Features

--------------------------

In [9]:
if features == 'base':
    X = data[['LATITUDE_meters', 
              'LONGITUDE_meters', 
              'Velocity', 
              'Angle_V',
              'dist_Tbar']]
elif features == 'extend':
    X = data[['LATITUDE_meters', 
              'LONGITUDE_meters', 
              'Velocity', 
              'Angle_V',
              'Acceleration', 
              'Angle_A',
              'dist_Tbar']]    
elif features == 'wind':
    X = data[['LATITUDE_meters', 
              'LONGITUDE_meters', 
              'Velocity', 
              'Angle_V',
              'Velocity_p',
              'dist_Tbar']]
    
y = data['Target_sec']

---------------------
# Random Forest Model

---------------------


In [10]:
model_rfr = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [11]:
model_rfr.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

-------------
## Error - Random Forest Model
-------------

In [12]:
y_pred = model_rfr.predict(X)

In [13]:
# The mean squared error
print('Mean absolute error Test: %.2f' % mean_absolute_error(y, y_pred) + ' seconds')

Mean absolute error Test: 16.68 seconds


**NOTA:** El Mean absolute error Test que se obtiene aquí debe estar muy cercano al que se obtiene al ejecutar el Notebook correspondiente a dicho modelo.

Los valores no son identicos debido a que no se toma siempre los mismo vuelos para el entrenamiento del modelo.

**El número máximo de vuelos en este dataset** 

In [14]:
print('Número de vuelos en el dataset: ', max_fligts)

Número de vuelos en el dataset:  1892


----------------
## Save model

----------------

In [15]:
if save == 1:
    joblib.dump(model_rfr, path_save  + name_model + '_' + model_type + '_' + features + '.pkl') 
